In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os
import pandas as pd

class UserPhotoDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, max_photos=40):
        self.user_data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.max_photos = max_photos

    def __len__(self):
        return len(self.user_data)

    def __getitem__(self, idx):
        user_row = self.user_data.iloc[idx]
        user = user_row['username']
        labels = user_row[['O', 'C', 'E', 'A', 'N']].values.astype('float32')
        user_dir = os.path.join(self.root_dir, user)

        if not os.path.exists(user_dir):
            print(f"Warning: User directory {user_dir} not found. Skipping this user.")
            return None

        photos = [os.path.join(user_dir, photo) for photo in os.listdir(user_dir)]

        # Truncate or pad photos to max_photos
        if len(photos) > self.max_photos:
            photos = photos[:self.max_photos]
        else:
            photos += [photos[0]] * (self.max_photos - len(photos))  # Repeat the first photo

        images = []
        for photo in photos:
            image = Image.open(photo)
            if self.transform:
                image = self.transform(image)
            images.append(image)

        images = torch.stack(images, dim=0)
        labels = torch.tensor(labels)
        return images, labels

    def __iter__(self):
        for idx in range(len(self)):
            item = self.__getitem__(idx)
            if item is not None:
                yield item


In [ ]:
import torchvision.models as models
from torchvision.models import ResNet18_Weights

class UserClassifier(nn.Module):
    def __init__(self, num_labels):
        super(UserClassifier, self).__init__()
        self.feature_extractor = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.feature_extractor.fc = nn.Identity()  # Remove the final fully connected layer
        self.fc = nn.Linear(512, num_labels)  # Assuming ResNet18 and 512 features output

    def forward(self, x):
        # x is of shape (batch_size, num_photos, channels, height, width)
        batch_size, num_photos, channels, height, width = x.shape
        x = x.view(batch_size * num_photos, channels, height, width)
        features = self.feature_extractor(x)
        features = features.view(batch_size, num_photos, -1)
        aggregated_features = torch.mean(features, dim=1)
        out = self.fc(aggregated_features)
        return out

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = UserPhotoDataset(csv_file='/content/drive/MyDrive/PFA-Dataset/datas/filtered_final_data.csv',
                           root_dir='/content/drive/MyDrive/PFA-Dataset/Images', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)


model = UserClassifier(num_labels=5)  # 5 binary labels
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(dataloader):
        print(f"Batch {i+1}/{len(dataloader)}")
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}')

In [ ]:
torch.save(model.state_dict(), '/content/drive/Shareddrives/PFA_Dataset/models/cnnUser.pth')

In [ ]:
model = UserClassifier(num_labels=5)  # Ensure that the architecture matches
model.load_state_dict(torch.load('/content/drive/MyDrive/PFA-Dataset/models/cnnUser.pth'))
model.eval()  # Set the model to evaluation mode


UserClassifier(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

### Evaluating the model

In [ ]:

dataset = UserPhotoDataset(
    csv_file='/content/drive/MyDrive/PFA-Dataset/datas/filtered_final_data.csv',
    root_dir='/content/drive/MyDrive/PFA-Dataset/Images',
    transform=transform
)

# Filter out None values in the DataLoader
dataloader = DataLoader(
    [item for item in dataset if item is not None],
    batch_size=4, shuffle=True, num_workers=2
)


In [ ]:

import torch
import numpy as np
from sklearn.metrics import classification_report, hamming_loss, confusion_matrix

# Ensure the model is in evaluation mode
model.eval()

# Variables to store all labels and predictions
all_labels = []
all_preds = []

# Disable gradient calculation for inference
with torch.no_grad():
    for images, labels in dataloader:
        # Get model predictions
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions

        # Store labels and predictions
        all_labels.append(labels.cpu().numpy())
        all_preds.append(preds.cpu().numpy())

# Concatenate all predictions and labels
all_labels = np.vstack(all_labels)
all_preds = np.vstack(all_preds)

# Print classification report
print("Classification Report")
print(classification_report(all_labels, all_preds, target_names=['O', 'C', 'E', 'A', 'N'], zero_division=0))

# Calculate and print Hamming Loss
ham_loss = hamming_loss(all_labels, all_preds)
print(f'Hamming Loss: {ham_loss}')

# Print confusion matrix for each label
print("Confusion Matrix for Each Label")
for i, label in enumerate(['O', 'C', 'E', 'A', 'N']):
    cm = confusion_matrix(all_labels[:, i], all_preds[:, i])
    print(f'Confusion Matrix for {label}:\n{cm}\n')



Classification Report
              precision    recall  f1-score   support

           O       0.70      1.00      0.82        35
           C       0.58      1.00      0.73        29
           E       0.00      0.00      0.00        21
           A       0.67      0.73      0.70        33
           N       0.00      0.00      0.00         6

   micro avg       0.64      0.71      0.67       124
   macro avg       0.39      0.55      0.45       124
weighted avg       0.51      0.71      0.59       124
 samples avg       0.65      0.68      0.64       124

Hamming Loss: 0.34
Confusion Matrix for Each Label
Confusion Matrix for O:
[[ 0 15]
 [ 0 35]]

Confusion Matrix for C:
[[ 0 21]
 [ 0 29]]

Confusion Matrix for E:
[[28  1]
 [21  0]]

Confusion Matrix for A:
[[ 5 12]
 [ 9 24]]

Confusion Matrix for N:
[[44  0]
 [ 6  0]]


In [ ]:
model = UserClassifier(num_labels=5)
model.load_state_dict(torch.load('/content/drive/Shareddrives/PFA_Dataset/models/cnnUser.pth'))
print(model)

UserClassifier(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin